# EDA & Featurizing

In [1]:
import pandas as pd
train=pd.read_csv("/content/train_MpHjUjU.csv")
test=pd.read_csv("/content/test_hXY9mYw.csv")
sample=pd.read_csv("/content/sample_submission_znWiLZ4.csv")

In [2]:
train.head(5)

,MMM-YY,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaN,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
3,2017-11-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1
4,2017-12-01,2,31,Male,C7,Master,67016,2017-11-06,NaN,2,2,0,1


In [3]:
train['MMM-YY']=pd.to_datetime(train['MMM-YY'],format="%Y/%m/%d")
train.Dateofjoining=pd.to_datetime(train.Dateofjoining,format="%Y/%m/%d")
train.LastWorkingDate=pd.to_datetime(train.LastWorkingDate,format="%Y/%m/%d")

In [10]:
train.rename(columns = {'MMM-YY':'Reporting_date'}, inplace = True)

In [16]:
X=train

In [21]:
import datetime
target=[]
train_group=train.groupby(by='Emp_ID')
ind=0
for key, item in train_group:
  group=train_group.get_group(key)
  last_work_date=group.LastWorkingDate.iloc[-1]
  left=1
  if(len(str(last_work_date))==3): 
    left=0
  if(left==1):
    for i in range(len(group)):
      current_date=group.Reporting_date.iloc[i]
      if((last_work_date-current_date)<=datetime.timedelta(180)):
        target.append(1)
      else:
        target.append(0)
      ind+=1
  else:
    last_report_date=group.Reporting_date.iloc[-1]
    for i in range(len(group)):
      current_date=group.Reporting_date.iloc[i]
      if((last_report_date-current_date)>=datetime.timedelta(180)):
        target.append(0)  
      else:
        X=X.drop(ind)  
      ind+=1

In [23]:
X.shape,len(target)

((15467, 13), 15467)

In [25]:
X.head(5)

,Reporting_date,Emp_ID,Age,Gender,City,Education_Level,Salary,Dateofjoining,LastWorkingDate,Joining Designation,Designation,Total Business Value,Quarterly Rating
0,2016-01-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,2381060,2
1,2016-02-01,1,28,Male,C23,Master,57387,2015-12-24,NaT,1,1,-665480,2
2,2016-03-01,1,28,Male,C23,Master,57387,2015-12-24,2016-03-11,1,1,0,2
5,2016-12-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1
6,2017-01-01,4,43,Male,C13,Master,65603,2016-12-07,NaT,2,2,0,1


In [28]:
days_in_company=X.Reporting_date-X.Dateofjoining
len(days_in_company)

15467

In [33]:
experience=[]
for i in days_in_company:
  experience.append(int(str(i).split(' ')[0]))

In [36]:
X['Experience']=experience

In [37]:
from sklearn.preprocessing import LabelEncoder

gender_enc=LabelEncoder()
gender_enc.fit(X.Gender)
X['Gender_Encoded']=gender_enc.transform(X.Gender)

city_enc=LabelEncoder()
city_enc.fit(X.City)
X['City_Encoded']=city_enc.transform(X.City)

edu_enc=LabelEncoder()
edu_enc.fit(X.Education_Level)
X['Education_Encoded']=edu_enc.transform(X.Education_Level)

In [42]:
X_new=X[['Age','Gender_Encoded','City_Encoded','Education_Encoded','Salary','Joining Designation','Designation','Total Business Value','Quarterly Rating','Experience']]
X_new

,Age,Gender_Encoded,City_Encoded,Education_Encoded,Salary,Joining Designation,Designation,Total Business Value,Quarterly Rating,Experience
0,28,1,15,2,57387,1,1,2381060,2,8
1,28,1,15,2,57387,1,1,-665480,2,39
2,28,1,15,2,57387,1,1,0,2,68
5,43,1,4,2,65603,2,2,0,1,-6
6,43,1,4,2,65603,2,2,0,1,25
...,...,...,...,...,...,...,...,...,...,...
19093,28,0,12,2,69498,1,1,319740,2,224
19094,28,0,12,2,69498,1,1,0,1,255
19095,28,0,12,2,69498,1,1,0,1,285
19096,28,0,12,2,69498,1,1,0,1,316


In [47]:
X_new.index=range(15467)

In [49]:
x_train=X_new.iloc[:12000]
x_test=X_new.iloc[12000:]
y_train=target[:12000]
y_test=target[12000:]

#Featurizing Test Data

In [52]:
test_data=train.merge(test,on='Emp_ID')
test_data.shape

(8588, 13)

In [60]:
test_data['Gender_Encoded']=gender_enc.transform(test_data.Gender)
test_data['City_Encoded']=city_enc.transform(test_data.City)
test_data['Education_Encoded']=edu_enc.transform(test_data.Education_Level)

In [61]:
days_in_company=test_data.Reporting_date-test_data.Dateofjoining
experience=[]
for i in days_in_company:
  experience.append(int(str(i).split(' ')[0]))
test_data['Experience']=experience
len(experience)

8588

In [64]:
test_data_new=test_data[['Emp_ID','Age','Gender_Encoded','City_Encoded','Education_Encoded','Salary','Joining Designation','Designation','Total Business Value','Quarterly Rating','Experience']]

In [67]:
test_group=test_data_new.groupby(by='Emp_ID')
test_data_final=[]
for key,value in test_group:
  group=test_group.get_group(key)
  test_data_final.append(group.iloc[-1])

In [70]:
test_data_final=pd.DataFrame(test_data_final)
test_data_final.columns=['Emp_ID','Age','Gender_Encoded','City_Encoded','Education_Encoded','Salary','Joining Designation','Designation','Total Business Value','Quarterly Rating','Experience']
test_data_final.drop(['Emp_ID'],axis=1,inplace=True)
test_data_final

,Age,Gender_Encoded,City_Encoded,Education_Encoded,Salary,Joining Designation,Designation,Total Business Value,Quarterly Rating,Experience
1,31,1,26,2,67016,2,2,0,1,25
6,31,0,2,0,78728,3,3,0,2,123
7,28,0,10,2,42172,1,1,0,1,-6
10,39,0,18,1,19734,3,3,0,1,46
34,31,1,16,0,102077,1,3,2013180,4,1128
...,...,...,...,...,...,...,...,...,...,...
8544,27,1,28,1,85112,3,3,0,1,60
8545,35,1,4,2,50180,2,2,0,1,2
8556,25,1,15,2,46952,2,2,2366500,4,287
8580,34,1,16,1,82815,2,3,505480,4,1873


# Modelling - Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
n_estimator=[20,50,100,200]
depth=[5,20,50,100]
max_f1=0
best=(0,0)
for i in n_estimator:
  for j in depth:
    clf=RandomForestClassifier(max_depth=j,n_estimators=i)
    clf.fit(x_train,y_train)
    f1=f1_score(y_test,clf.predict(x_test))
    print(i,j,f1)
    if(max_f1<f1):
      max_f1=f1
      best=(i,j)
print(max_f1,best)

In [80]:
clf=RandomForestClassifier(max_depth=100,n_estimators=200)
clf.fit(X_new,target)
final_pred=clf.predict(test_data_final) 

In [82]:
test['Target']=final_pred
test

,Emp_ID,Target
0,394,1
1,173,0
2,1090,1
3,840,1
4,308,0
...,...,...
736,2134,1
737,2255,1
738,448,0
739,1644,0


In [83]:
test.to_csv('finalpred1.csv',index=False)

# Modelling- Neural Network

In [92]:
from keras import backend as K
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [221]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.preprocessing import sequence
import numpy as np
from keras.layers import Activation, Flatten, Dropout, Input, Concatenate
from keras import regularizers, optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from keras.layers.embeddings import Embedding

input_layer = Input(shape=(12))
conv_1= Dense(64, activation='relu',kernel_initializer='he_normal')(input_layer)
conv_2= Dense(32, activation='relu',kernel_initializer='he_normal')(conv_1)
output_layer = Dense(units=1,activation='sigmoid')(conv_2)  
model = Model(input_layer, output_layer)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=[f1_m])
model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 12)]              0         
                                                                 
 dense_39 (Dense)            (None, 64)                832       
                                                                 
 dense_40 (Dense)            (None, 32)                2080      
                                                                 
 dense_41 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [110]:
len(X_new),len(target)

(15467, 15467)

In [111]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_new,target,test_size=0.30,stratify=target)
print(x_train.shape)

(10826, 10)


In [112]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [120]:
model.fit(x_train,y_train,epochs=80,validation_data=(x_test,y_test),verbose=2)

Epoch 1/80
339/339 - 1s - loss: 5999.8018 - f1_m: 0.5140 - val_loss: 1097.4717 - val_f1_m: 0.6314 - 1s/epoch - 4ms/step
Epoch 2/80
339/339 - 1s - loss: 1127.2146 - f1_m: 0.4957 - val_loss: 394.3604 - val_f1_m: 0.6440 - 788ms/epoch - 2ms/step
Epoch 3/80
339/339 - 1s - loss: 1164.2487 - f1_m: 0.4988 - val_loss: 871.1063 - val_f1_m: 0.6531 - 790ms/epoch - 2ms/step
Epoch 4/80
339/339 - 1s - loss: 900.9874 - f1_m: 0.5087 - val_loss: 842.4277 - val_f1_m: 0.6863 - 609ms/epoch - 2ms/step
Epoch 5/80
339/339 - 1s - loss: 889.9233 - f1_m: 0.5083 - val_loss: 850.9545 - val_f1_m: 0.6120 - 621ms/epoch - 2ms/step
Epoch 6/80
339/339 - 1s - loss: 980.7723 - f1_m: 0.5084 - val_loss: 482.5494 - val_f1_m: 0.6732 - 783ms/epoch - 2ms/step
Epoch 7/80
339/339 - 1s - loss: 630.7247 - f1_m: 0.5147 - val_loss: 1820.6921 - val_f1_m: 0.3692 - 613ms/epoch - 2ms/step
Epoch 8/80
339/339 - 1s - loss: 793.0920 - f1_m: 0.5100 - val_loss: 268.1395 - val_f1_m: 0.5958 - 620ms/epoch - 2ms/step
Epoch 9/80
339/339 - 1s - loss

In [121]:
test_data_final.shape

(741, 10)

In [123]:
final_pred=model.predict(test_data_final) 
final_pred_target=[]
for i in final_pred:
  if(i>0.5):
    final_pred_target.append(1) 
  else:
    final_pred_target.append(0)

In [124]:
test['Target']=final_pred_target
test

,Emp_ID,Target
0,394,1
1,173,0
2,1090,1
3,840,1
4,308,0
...,...,...
736,2134,1
737,2255,1
738,448,0
739,1644,0


In [125]:
test.to_csv('finalpred2.csv',index=False)

# New Features

In [131]:
X_new['Prev_Business_Value']=X_new['Total Business Value'].shift(1)
X_new['Prev_Rating']=X_new['Quarterly Rating'].shift(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
for i in range(len(X_new)):
  if(np.isnan(X_new.Prev_Business_Value.iloc[i])):
    X_new.Prev_Business_Value.iloc[i]=X_new['Total Business Value'].iloc[i]
  if(np.isnan(X_new.Prev_Rating.iloc[i])):
    X_new.Prev_Rating.iloc[i]=X_new['Quarterly Rating'].iloc[i]

In [140]:
X_new

,Age,Gender_Encoded,City_Encoded,Education_Encoded,Salary,Joining Designation,Designation,Total Business Value,Quarterly Rating,Experience,Prev_Business_Value,Prev_Rating
0,28,1,15,2,57387,1,1,2381060,2,8,2381060.0,2.0
1,28,1,15,2,57387,1,1,-665480,2,39,2381060.0,2.0
2,28,1,15,2,57387,1,1,0,2,68,-665480.0,2.0
3,43,1,4,2,65603,2,2,0,1,-6,0.0,2.0
4,43,1,4,2,65603,2,2,0,1,25,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15462,28,0,12,2,69498,1,1,319740,2,224,250000.0,2.0
15463,28,0,12,2,69498,1,1,0,1,255,319740.0,2.0
15464,28,0,12,2,69498,1,1,0,1,285,0.0,1.0
15465,28,0,12,2,69498,1,1,0,1,316,0.0,1.0


In [184]:
test_data_new.shape

(8588, 11)

In [ ]:
test_data_new['Prev_Business_Value']=test_data_new['Total Business Value'].shift(1)
test_data_new['Prev_Rating']=test_data_new['Quarterly Rating'].shift(1)
for i in range(len(test_data_new)):
  if(np.isnan(test_data_new.Prev_Business_Value.iloc[i])):
    test_data_new.Prev_Business_Value.iloc[i]=test_data_new['Total Business Value'].iloc[i]
  if(np.isnan(test_data_new.Prev_Rating.iloc[i])):
    test_data_new.Prev_Rating.iloc[i]=test_data_new['Quarterly Rating'].iloc[i]

In [187]:
test_data_new

,Emp_ID,Age,Gender_Encoded,City_Encoded,Education_Encoded,Salary,Joining Designation,Designation,Total Business Value,Quarterly Rating,Experience,Prev_Business_Value,Prev_Rating
0,2,31,1,26,2,67016,2,2,0,1,-5,0.0,1.0
1,2,31,1,26,2,67016,2,2,0,1,25,0.0,1.0
2,6,31,0,2,0,78728,3,3,0,1,1,0.0,1.0
3,6,31,0,2,0,78728,3,3,0,1,32,0.0,1.0
4,6,31,0,2,0,78728,3,3,0,2,62,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8583,2788,30,1,19,2,70254,2,2,740280,3,54,497690.0,3.0
8584,2788,30,1,19,2,70254,2,2,448370,3,85,740280.0,3.0
8585,2788,30,1,19,2,70254,2,2,0,2,115,448370.0,3.0
8586,2788,30,1,19,2,70254,2,2,200420,2,146,0.0,2.0


In [188]:
test_group=test_data_new.groupby(by='Emp_ID')
test_data_final2=[]
for key,value in test_group:
  group=test_group.get_group(key)
  test_data_final2.append(group.iloc[-1])

In [194]:
test_data_final2=pd.DataFrame(test_data_final2,columns=test_data_new.columns)
test_data_final2.drop(['Emp_ID'],axis=1,inplace=True)
test_data_final2.shape

(741, 12)

# Modelling on new features(NN)

In [222]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_new,target,test_size=0.30,stratify=target)
print(x_train.shape)
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=[f1_m])
model.fit(x_train,y_train,epochs=70,validation_data=(x_test,y_test),verbose=2)

(10826, 12)
Epoch 1/70
339/339 - 1s - loss: 28487.3281 - f1_m: 0.5670 - val_loss: 1870.8218 - val_f1_m: 0.5448 - 1s/epoch - 4ms/step
Epoch 2/70
339/339 - 1s - loss: 3069.7878 - f1_m: 0.5559 - val_loss: 5591.3745 - val_f1_m: 0.5020 - 806ms/epoch - 2ms/step
Epoch 3/70
339/339 - 1s - loss: 3446.0669 - f1_m: 0.5462 - val_loss: 5397.9614 - val_f1_m: 0.6336 - 672ms/epoch - 2ms/step
Epoch 4/70
339/339 - 1s - loss: 3254.9736 - f1_m: 0.5330 - val_loss: 2616.7405 - val_f1_m: 0.6475 - 766ms/epoch - 2ms/step
Epoch 5/70
339/339 - 1s - loss: 2725.6477 - f1_m: 0.5258 - val_loss: 13369.6328 - val_f1_m: 0.6387 - 620ms/epoch - 2ms/step
Epoch 6/70
339/339 - 1s - loss: 3668.2253 - f1_m: 0.5437 - val_loss: 4757.8145 - val_f1_m: 0.5010 - 764ms/epoch - 2ms/step
Epoch 7/70
339/339 - 1s - loss: 2662.7681 - f1_m: 0.5536 - val_loss: 2454.2329 - val_f1_m: 0.6459 - 629ms/epoch - 2ms/step
Epoch 8/70
339/339 - 1s - loss: 2274.9326 - f1_m: 0.5356 - val_loss: 12947.8408 - val_f1_m: 0.5430 - 604ms/epoch - 2ms/step
Epoc

In [229]:
model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test),verbose=2)

339/339 - 1s - loss: 6.1523 - f1_m: 0.6460 - val_loss: 2.6668 - val_f1_m: 0.6917 - 635ms/epoch - 2ms/step


In [230]:
model.save('model.h5')

In [231]:
final_pred=model.predict(test_data_final2) 
final_pred_target=[]
for i in final_pred:
  if(i>0.5):
    final_pred_target.append(1)
  else:
    final_pred_target.append(0)

In [232]:
test['Target']=final_pred_target
test

,Emp_ID,Target
0,394,1
1,173,0
2,1090,1
3,840,1
4,308,0
...,...,...
736,2134,1
737,2255,1
738,448,0
739,1644,1


In [233]:
test.to_csv('finalpred6.csv',index=False)

# Random on new dataset

In [204]:
X_new.shape
#len(target)

(15467, 12)

In [205]:
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':[20,50,100,200,500], 'max_depth':[5,10,20,30,50,100]}
est=RandomForestClassifier()
clf = GridSearchCV(est, parameters)
clf.fit(X_new,target)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 20, 30, 50, 100],
                         'n_estimators': [20, 50, 100, 200, 500]})

In [206]:
clf.best_params_

{'max_depth': 5, 'n_estimators': 100}

In [234]:
ran=RandomForestClassifier(max_depth=5,n_estimators=100)
ran.fit(X_new,target)

RandomForestClassifier(max_depth=5)

In [235]:
f1_score(y_test,ran.predict(x_test))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


0.7467113130829945

In [236]:
final_pred=ran.predict(test_data_final2) 
test['Target']=final_pred


In [237]:
test

,Emp_ID,Target
0,394,1
1,173,0
2,1090,1
3,840,1
4,308,0
...,...,...
736,2134,1
737,2255,1
738,448,0
739,1644,0


In [238]:
test.to_csv('finalpred7.csv',index=False)